In [1]:
!pip install -q telebot
!pip install -q pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 kB 6.6 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import tensorflow as tf

import pydub
import telebot

In [3]:
mybot = telebot.TeleBot("8117558510:AAHC4sNSrozmZC8uCEXIM1BEfMZez1iwWRE", parse_mode=None)
labels = ['Abdollah', 'Azra', 'Davood', 'Javad', 'Khadijeh', 'Kiana', 'Mahdi', 'Maryam', 'Matin', 'Melika', 'Mohadeseh', 'Mohammad', 'Mohammad_parvari', 'Mona', 'Nahid', 'Nima', 'Omid', 'Parisa', 'Parsa', 'Sajedeh', 'Shima', 'Tara']
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab_Notebooks/PyLearn7-Assignment59/AudioClassification/weights/audio_classification.h5')


In [4]:
os.makedirs("voice")
os.makedirs("input_audio")
os.makedirs("input_audio_chunks")

In [ ]:
@mybot.message_handler(commands=['start'])
def send_welcome(message):
    msg = mybot.send_message(message.chat.id,"Hi "+str(message.chat.first_name) + "\n" +
                             "Welcome to 'pydio classifier bot'" + "\n" +
                             "If your name is in the list below, this bot can recognize your sound." + "\n" +
                             "[Abdollah, Azra, Davood, Javad, Khadijeh, Kiana, Mahdi, Maryam,Matin, Melika, Mohadeseh, Mohammad, Mohammad_parvari, Mona, Nahid,Nima, Omid, Parisa, Parsa, Sajedeh, Shima, Tara]" + "\n" +
                             "/start_audio_recognition")


@mybot.message_handler(commands=['start_audio_recognition'])
def send_audio(message):
    msg = mybot.reply_to(message,"Please send me a few seconds of your voice.")
    mybot.register_next_step_handler(msg,recognize_audio)


def audio_preprocess(file_path):
    audio = pydub.AudioSegment.from_file(file_path)

    audio = audio.set_sample_width(2)    # convert to 16-bit (2 bytes per sample) to avoid tensorflow error
    audio = audio.set_frame_rate(48000)
    audio = audio.set_channels(1)        # convert stereo audio to mono audio to avoid tensorflow error

    chunks = pydub.silence.split_on_silence(audio, min_silence_len=2000, silence_thresh=-45)
    result = sum(chunks)
    result.export("input_audio.wav", format="wav")

    audio = pydub.AudioSegment.from_file("input_audio.wav")
    chunks = pydub.utils.make_chunks(audio, 1000)

    for i , chunk in enumerate(chunks):
        if len(chunk) < 1000 :
            continue
        chunk.export(os.path.join("input_audio_chunks", f"voice{i}.wav"), format="wav")

    audio_chunks = []
    for audio_chunk in os.listdir("input_audio_chunks"):
        audio_chunks.append(audio_chunk)

    xs = []
    for audio_chunk in audio_chunks:
        x = tf.io.read_file("input_audio_chunks/"+audio_chunk)
        x, sample_rate = tf.audio.decode_wav(x, desired_channels=1, desired_samples=48000)
        x = tf.squeeze(x, axis=-1)
        x = x[tf.newaxis,...]
        xs.append(x)

    return xs


def recognize_audio(message):

    fileID = message.voice.file_id
    file_info = mybot.get_file(fileID)
    file_path = file_info.file_path
    downloaded_file = mybot.download_file(file_path)

    with open(file_path, 'wb') as new_file:
        new_file.write(downloaded_file)

    xs = audio_preprocess(file_path)

    preds = []
    for x in xs:
        pred = model(x)
        label = np.argmax(pred)
        preds.append(labels[label])
    # print(preds)
    most_common = max(set(preds), key=preds.count)                # Finds the most common element in preds list
    # print(most)
    print("The voice belongs to: ", most_common)

    mybot.reply_to(message, f'The voice belongs to {most_common}')


mybot.infinity_polling()